# Assignment - Bangkok Airbnb dataset (15%, due 9th Jan 11.59 pm.)
Explore this Bangkok Airbnb dataset with Pandas on GoogleColab and answer these questions below. Note that every answer to the questions must accompany by python code displaying how to reach that answer. Failure to display the python code will result in a score deduction.

File 1) calendar.csv - The calendar file records the price, availability, and other details in properties.
*   listing_id : Airbnb's unique identifier for the listing
*   date : The date in the listing's calendar
*   available : Whether the date is available for a booking [boolean T/F]
*   price : The price (baht) listed for the day.

File 2) listings.csv - The listings file includes essential information regarding properties.
*  id : Airbnb's unique identifier for the listing
* name : Name of the listing
* beds : a number of bedrooms in the listing,
* host_name: Name of the host. Usually, just the first name(s)
* neighbourhood_cleansed: a district name in Bangkok where the listing is.
* amenities: a desirable or useful feature(s) on the listing. This column is in a list of string format.
* property_type: Self-selected property type.

Answer the questions below.
1. [xx, yy] What is the number of listing(rows) in calendar.csv and listings.csv?
2. How many of the listing properties with equal two or more beds are in the "Bang Sue" district?
3. How many unique items in "amenities" are shown in the listing? What are the top three most popular "amenities"?
4. How many listing records are in Sep 2022 on carlendar.csv? The same listing_id on different dates is considered a different record.
5. What is the highest average price on the listing? The average price is the average price of the same listing_id in all records/rows on the carlendar.csv.  
6. How many listing_id(s) do we observe where the lowest and highest prices differ more than 10,000 baht?
7. If we consider this price difference in 6. as malicious activity and want to remove them as outliers using the z-score statistical method. How many listing_id(s) do we observe as outliers?
An outlier threshold value is 3.0.
Hint 7. a) https://www.geeksforgeeks.org/detect-and-remove-the-outliers-using-python/
Hint 7. b) use can apply the NumPy function directly to Pandas "Series" as a new column.
8. Show malicious host_name from 7.
9. Find the number of listings for each host from 7.

-------From 10 - 15, please remove malicious listing_id in the calculation. ------  
10. Find The top 3 districts with the most listings.  
11. Find the top 3 most common property_type.  
12. If we are looking for a property with two bedrooms in the Parthum Wan district,  how many options(listing_id) do we have?    
13. From 12, what is the price range on those options? Use the average price as the price of a listing property.    
14. If we are looking for a property with two bedrooms in the Parthum Wan district and has "review_scores_rating" >= 3,  how many options (listing_id) do we have?  
15. From 14, what is the price average of listing properties on question 14 conditions (two bedrooms in the Parthum Wan district and has "review_scores_rating" >= 3)  
* hint: average on average is taboo.
  
Please submit your colab worksheet to google classroom by 9th Jan 11.59 pm. Late submission will deduct 1% per day.


In [56]:
import pandas as pd
from scipy import stats
import numpy as np
from ast import literal_eval
import matplotlib.pyplot as plt
import os
os.chdir('/Users/fulinq/Documents/KMITL/FinancialEngineering/Y2/T2/DataDataSci')
pd.set_option('display.max_columns', 74)
pd.set_option('display.width', 500)

# !gdown 1d0yPTxU6Q2ae6m0N9a9z2GEOZXbgErKY
# !gdown 1cGJGnacPmbvQJ-lNeqaYp27BYFDbThEQ

df_Carl = pd.read_csv('calendar.csv')
df_List = pd.read_csv('listings.csv')

df_List["amenities"]= df_List['amenities'].apply(lambda x: literal_eval(str(x)))


Q1)  the number of calendar.csv and listings.csv are 6231645 and 17073, repectively.

In [57]:
print(len(df_Carl),len(df_List)) #1

6231645 17073


Q2) the number of properties with equal two or more beds are in the "Bang Sue" district is 73.

In [58]:
print(len(df_List[(df_List['beds'] >= 2) & (df_List['neighbourhood_cleansed'] == 'Bang Sue')])) #2

73


Q3) 
* There are 993 items in "amenities" showing in the listing.
* The top three most popular "amenities" are Long term stays allowed, Air conditioning, Wifi.


In [59]:
countDict = dict()
for lst in df_List['amenities']:
    for item in lst:
        countDict[item] = countDict.get(item, 0) + 1
        
countDict = sorted(countDict.items(), key = lambda x : x[1], reverse = True)
print(f'Unique amenities : {len(countDict)}')
print('top 3 most popular amenities : ')
for item in countDict[0:3]:
    print(f'\t{item[0]} : {item[1]} items')


Unique amenities : 993
top 3 most popular amenities : 
	Long term stays allowed : 16168 items
	Air conditioning : 15883 items
	Wifi : 14496 items


Q4) there are 512190 listing records are in Sep 2022 on carlendar.csv.

In [60]:
df_Carl['date'] = pd.to_datetime(df_Carl['date'], yearfirst=True)
result = len(df_Carl[(df_Carl['date'] >= '2022-08-01') & (df_Carl['date'] <= '2022-08-30')])
print(f'Number of records are in Sep 2022 on carlendar.csv : {result} records')                 #4

Number of records are in Sep 2022 on carlendar.csv : 512190 records


Q5) the highest average price on the listing is 1417827.397260274 baht

In [61]:
result = df_Carl.groupby('listing_id')['price'].mean().max()
print(f'the highest average price on the listing : {result} baht')

the highest average price on the listing : 1417827.397260274 baht


Q6) There are 43 "listing_id" having prices differ more than 10,000 baht.

In [62]:
selectedDf = df_Carl.groupby('listing_id').agg({'price' : ['min','max']})
selectedDf['differ'] = selectedDf.apply(lambda x: x[('price', 'max')] - x[('price', 'min')], axis=1)
result = selectedDf[selectedDf['differ'] > 10000]
print(f'Number of "listing_id" having prices differ more than 10,000 : {len(result)} lists')

Number of "listing_id" having prices differ more than 10,000 : 43 lists


Q7) If we consider this price difference in 6. as malicious activity and want to remove them as outliers using the z-score statistical method. How many listing_id(s) do we observe as outliers?
An outlier threshold value is 3.0.
Hint 7. a) https://www.geeksforgeeks.org/detect-and-remove-the-outliers-using-python/
Hint 7. b) use can apply the NumPy function directly to Pandas "Series" as a new column.

In [63]:
selectedDf = df_Carl.groupby('listing_id').agg({'price' : ['min','max']})
selectedDf['differ'] = selectedDf.apply(lambda x: x[('price', 'max')] - x[('price', 'min')], axis=1)
# print(result)
z = np.abs(stats.zscore(selectedDf['differ']))


#outlier threshold = 3.0
Outlier = np.where(z > 3) 
# print(Outlier)
print(f'Number of outlier : {len(Outlier[0])}')

Number of outlier : 12


Q8) Show malicious host_name from 7.
* Keiko And Pete
* Nuttawut
* Pongwich
* FoRest
* Oliver
* Rohan
* May

In [64]:
mer_df = df_Carl.groupby('listing_id').agg({'price' : ['min','max', 'mean']})
mer_df['differ'] = mer_df.apply(lambda x: x[('price', 'max')] - x[('price', 'min')], axis=1)
z = np.abs(stats.zscore(mer_df['differ']))
mer_df['Z-Score'] = z
mer_df.columns = mer_df.columns.map('_'.join)
mer_df.reset_index(inplace=True)
# print(mer_df)

selectedDf = mer_df.merge(df_List, left_on='listing_id', right_on='id', how='left')
selectedDf = selectedDf.drop_duplicates(subset='listing_id')
result = selectedDf.iloc[Outlier[0]]
# print(selectedDf.info())
print(result['host_name'].to_string(index=False))

Keiko And Pete
      Nuttawut
      Pongwich
      Nuttawut
      Nuttawut
      Nuttawut
        FoRest
      Nuttawut
        Oliver
         Rohan
         Rohan
           May


Q9) number of listings for each host from 7:
* FoRest   : 3  room(s)
* May   : 19  room(s)
* Nuttawut   : 5  room(s)
* Keiko And Pete   : 1  room(s)
* Rohan   : 31  room(s)
* Oliver   : 2  room(s)
* Pongwich   : 2  room(s)

In [65]:
## Previous data
# mer_df = df_Carl.groupby('listing_id').agg({'price' : ['min','max']})
# mer_df['differ'] = mer_df.apply(lambda x: x[('price', 'max')] - x[('price', 'min')], axis=1)
# z = np.abs(stats.zscore(mer_df['differ']))

# selectedDf = df_Carl.merge(df_List, left_on='listing_id', right_on='id', how='left')
# selectedDf = selectedDf.drop_duplicates(subset='listing_id')
# result = selectedDf.iloc[Outlier[0]]

nameSet = set(result['host_name'].tolist())
counter = dict()
for name in nameSet:
    counter[name] = counter.get(name, selectedDf[selectedDf['host_name'] == name].shape[0])

for item in counter:
    print(f'{item}   : {counter[item]}  room(s)')

FoRest   : 3  room(s)
May   : 19  room(s)
Nuttawut   : 5  room(s)
Keiko And Pete   : 1  room(s)
Rohan   : 31  room(s)
Oliver   : 2  room(s)
Pongwich   : 2  room(s)


# removing malicious listing_id in the calculation.

In [66]:
print(f'Row before removing : {len(selectedDf)}')
removedList = selectedDf[~selectedDf['host_name'].isin(nameSet)]
print(f'Row after removing : {len(removedList)}')

Row before removing : 17073
Row after removing : 17010


Q10) The top 3 districts with the most listings are Vadhana, Khlong Toei, and Huai Khwang.

In [67]:
print(removedList['neighbourhood_cleansed'].value_counts().head(3))

neighbourhood_cleansed
Khlong Toei    2102
Vadhana        2092
Huai Khwang    1178
Name: count, dtype: int64


Q11) The top 3 most common property_type are Entire rental unit, Entire condo, and Private room in rental unit.  

In [68]:
print(removedList['property_type'].value_counts().head(3))

property_type
Entire rental unit             4039
Entire condo                   3641
Private room in rental unit    1773
Name: count, dtype: int64


Q12) There are 88 options with two bedrooms in the Parthum Wan district.

In [69]:
selectedDf = removedList[(removedList['neighbourhood_cleansed'] == 'Parthum Wan') & (removedList['beds'] == 2)]
print(f'The number of property with two bedrooms in the Parthum Wan district : {len(selectedDf)}')

The number of property with two bedrooms in the Parthum Wan district : 88


Q13) From 12, what is the price range on those options? Use the average price as the price of a listing property. 

In [70]:
minPrice = selectedDf['price_min'].min()
maxPrice = selectedDf['price_max'].max()
meanPrice = selectedDf['price_mean'].mean()
print(f'Price Range : {minPrice} - {maxPrice} THB\nAverage Price : {meanPrice} THB')

Price Range : 600.0 - 17458.0 THB
Average Price : 2667.94903486924 THB


Q14) There are 49 properties with two bedrooms in the Parthum Wan district and has "review_scores_rating" >= 3.

In [71]:
selectedDf = selectedDf[selectedDf["review_scores_rating"] >= 3]
print(len(selectedDf))

49


Q15) From 14, the average price is 2704.221023203802 THB

In [73]:
meanPrice = selectedDf['price_mean'].mean()
print(f'Average Price : {meanPrice} THB')

Average Price : 2704.221023203802 THB
